In [1]:
import os
import sys
import pickle

In [2]:
# add udtools library to path
home_dir = os.environ.get('HOME')
udtools_module_path = os.path.join(home_dir, 'git', 'ud-tools-core', 'udtools')

if udtools_module_path not in sys.path:
  sys.path.append(udtools_module_path)

In [3]:
import ipywidgets as widgets

# OCC/pythonocc-core imports
from OCC.Display.WebGl.jupyter_renderer import JupyterRenderer

# udtools imports
from archtools.geometry.boolean import diff
from nyczoning.zoninglot import ZoningLot
from nyczoning.geometry import build_yard_cutter
from nyclanduse.site import Site
from nyclanduse.scenario import Scenario
from nyclanduse.study import Study

Each of the Zone classes generated with ZoneFactory provides its own `get_additives` and `get_subtractives` method. In the case of `ContextualDistrict`, `get_additives` is not yet implemented. `get_subtractives` in this case first calls `build_height_cutter` to get limiting cutters used to trim down the base volume to the maximum height. Then it iterates over each list of lot edges on the zoning lot, using them as inputs to `build_yard_cutter`.

When `generate_envelope` is called, the ZoningLot object uses `build_base_volume` to get the unrestricted volumetric area on the zoning lot. Then it calls `get_additives` and `get_subtractives` on the current Zone object (`zone`) to get volumes to either add or subtract through boolean operations to obtain the finished zoning envelope.

Ideally, `build_yard_cutter` would take an edge and the OCC Wire representation of the boundary, and a series of height/setback pairs used to define the contour of the cutting volume.

In the cell below, provide one or more BBLs to use as the zoning lot, separated by commas. For example:

- `2022850129, 2022850130`
- `2022850130`

In [4]:
bbl_form = widgets.Text(
    value='2022850130',
    placeholder='',
    description='BBL(s)',
    disabled=False
)

bbl_form

Text(value='2022850130', description='BBL(s)', placeholder='')

In [22]:
study = Study('WITH_ACTION')
study.add_scenario(Scenario('A'))

site_bbls = [s.strip() for s in bbl_form.value.split(',')]
study.add_site(Site('1', site_bbls))

zl = study.get_zoning_lot('A', '1')
zl.define_zone('R10X')

In [6]:
# zl.zone.ht_max = 105
# zl.zone.base_ht_max = 65

In [7]:
# zl.baseplane_elevation = 30

In [23]:
(subtractives, results) = zl.generate_envelope()
renderer = JupyterRenderer()
renderer.DisplayShape(zl.geom, render_edges=True, topo_level="default", shape_color="#abdda4", update=True)
renderer.DisplayShape(zl.envelope)

In [24]:
bldg = zl.generate_building()

In [10]:
bldg.floors

In [11]:
# bldg_summary = bldg.summarize()

In [25]:
for f in bldg.floors:
    print(f'Floor {f.level} | Area: {f.area} | Height: {f.height}')
    renderer.DisplayShape(f.geom, render_edges=True, shape_color="#abdda4")

Floor 1 | Area: 2178.7957450122185 | Height: 15.0
Floor 2 | Area: 1536.1335388175137 | Height: 10.0
Floor 3 | Area: 1536.1335388175137 | Height: 10.0
Floor 4 | Area: 1536.1335388175137 | Height: 10.0
Floor 5 | Area: 1536.1335388175137 | Height: 10.0
Floor 6 | Area: 1536.1335388175137 | Height: 10.0
Floor 7 | Area: 1536.1335388175137 | Height: 10.0
Floor 8 | Area: 1536.1335388175137 | Height: 10.0
Floor 9 | Area: 1214.8033653074197 | Height: 10.0
Floor 10 | Area: 1214.8033653074197 | Height: 10.0
Floor 11 | Area: 1214.8033653074197 | Height: 10.0
Floor 12 | Area: 1214.8033653074197 | Height: 10.0
Floor 13 | Area: 1214.8033653074197 | Height: 10.0
Floor 14 | Area: 1214.8033653074197 | Height: 10.0
Floor 15 | Area: 1214.8033653074197 | Height: 10.0
Floor 16 | Area: 1214.8033653074197 | Height: 10.0
Floor 17 | Area: 1214.8033653074197 | Height: 10.0
Floor 18 | Area: 1214.8033653074197 | Height: 10.0
Floor 19 | Area: 1214.8033653074197 | Height: 10.0
Floor 20 | Area: 1214.8033653074197 | He

## Checklist

- [x] `build_height_cutter` working
- [x] `build_yard_cutter` working
    - [x] implemented for rear yards
    - [x] implemented for front yards
    - [ ] implemented for side yards

In [13]:
# export zoning lot, edges & bounds

# with open('./assets/20210921_edge850.pickle', 'wb') as f:
#   pickle.dump(zl.edges[850]['geom'], f)
  
# with open('./assets/20210921_bounds2022850130.pickle', 'wb') as f:
#   pickle.dump(zl.geom, f)

# with open('./assets/20210928_zl.pickle', 'wb') as f:
#   pickle.dump(zl, f)